In [29]:
import torchaudio

In [60]:
audio, og_sample_rate = torchaudio.load('../audio_samples/puppy.mp3')

In [61]:
max_duration = 60
audio = audio[:,0:max_duration*og_sample_rate]
audio = torchaudio.transforms.Resample(og_sample_rate, 16000)(audio)